# Current state

# Managing adsorption film workflows

In [1]:
import fwtools

In [2]:
fwtools.auxiliary.printurn

AttributeError: module 'fwtools' has no attribute 'auxiliary'

In [3]:
fwtools.auxiliary.printurn(1)

AttributeError: module 'fwtools' has no attribute 'auxiliary'

In [4]:
%load_ext autoreload

%autoreload 2

%config Application.log_level="WARN"

## Init

In [5]:
from sys import path

In [6]:
path

['/opt/apps/fireworks/1.8.7/venv/fireworks-python-3.6/bin',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/opt/apps/fireworks/1.8.7/venv/fireworks-python-3.6/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/testuser/.ipython']

In [7]:
import sys

In [8]:
print( sys.executable )

/opt/apps/fireworks/1.8.7/venv/fireworks-python-3.6/bin/python


In [9]:
# tune noteboook width to desired value
# the margin-left option is necessary if toc is displayed on left hand side
from IPython.core.display import display, HTML
# good for toc on left side:
# display(HTML("<style>.container { width:85%; margin-left:15%; margin-right:0% !important; }</style>"))
display(HTML("<style>.container { width:100%; margin-left:0%; margin-right:0% !important; }</style>"))

In [10]:
import logging
import os
import datetime
from pprint import pprint

In [11]:
logging.getLogger().handlers #Delete Jupyter notebook root logger handler
logger = logging.getLogger()
logger.handlers = []

#Create logger as usual
logger = logging.getLogger(__name__)
logger.setLevel(logging.WARN)

In [12]:
logging.basicConfig(level=logging.WARN)

In [13]:
import pandas as pd

In [14]:
from fireworks import Firework, Workflow
from fireworks.user_objects.firetasks.jlh_tasks import DummyParentTask, RecoverPackmolTask

In [15]:
import fireworks.user_objects.firetasks.jlh_tasks
print( fireworks.user_objects.firetasks.jlh_tasks.__file__)

/opt/apps/fireworks/1.8.7/venv/fireworks-python-3.6/lib/python3.6/site-packages/fireworks/user_objects/firetasks/jlh_tasks.py


In [16]:
import fireworks.user_objects.firetasks.script_task
print( fireworks.user_objects.firetasks.script_task.__file__)

/opt/apps/fireworks/1.8.7/venv/fireworks-python-3.6/lib/python3.6/site-packages/fireworks/user_objects/firetasks/script_task.py


In [17]:
print( fireworks.user_objects.firetasks.script_task.__file__)

/opt/apps/fireworks/1.8.7/venv/fireworks-python-3.6/lib/python3.6/site-packages/fireworks/user_objects/firetasks/script_task.py


In [18]:
from fwtools.JobAdmin import JobAdmin
from fwtools.JobAdmin import l_CTAB, head_atom_number_CTAB, tail_atom_number_CTAB
from fwtools.JobAdmin import l_SDS, head_atom_number_SDS, tail_atom_number_SDS

In [19]:
prefix = os.getcwd()

In [20]:
template_prefix = prefix + os.sep + 'N_surfactant_on_substrate_template'

In [21]:
output_prefix = prefix + os.sep + 'sys'

In [22]:
# minimal JobAdmin instance
jobadmin = JobAdmin( 
    queue   = 'BWCLOUD',
    template_prefix = template_prefix,
    output_prefix   = output_prefix
)

In [23]:
# jobadmin.fp.reset()

In [24]:
# jobadmin.fp.add_file(
#    path="/mnt/dat/work/testuser/md/surfactants/data/surfactant_on_AU_111.json",
#    identifier='surfactant_on_AU_111_df_json')

In [25]:
# insert scripts and templates into db

In [26]:
jobadmin.fp.add_file(
    template_prefix + os.sep + 'jlh_vmd.tcl',
    identifier='jlh_vmd.tcl',
    metadata={
        'type':     'script',
        'language': 'tcl',
        'usecase':  'indenter insertion in vmd'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'indenter_insertion.tcl',
    identifier='indenter_insertion.tcl',
    metadata={
        'type':     'template',
        'language': 'tcl',
        'usecase':  'indenter insertion in vmd'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'MergeLammpsDataFiles.py',
    identifier='MergeLammpsDataFiles.py',
    metadata={
        'type':     'script',
        'language': 'python',
        'usecase':  'merge VMD output with parameters from original system'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'merge.py',
    identifier='merge.py',
    metadata={
        'type':     'script',
        'language': 'python',
        'usecase':  'merge VMD output with parameters from original system'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'extract_thermo.sh',
    identifier='extract_thermo.sh',
    metadata={
        'type':     'script',
        'language': 'bash',
        'usecase':  'extract stats from LAMMPS log'})

jobadmin.fp.delete_file(identifier='lmp_header.input')

jobadmin.fp.add_file(
    template_prefix + os.sep + 'lmp_header.input',
    identifier='lmp_header.input',
    metadata={
        'type':     'input',
        'language': 'LAMMPS',
        'usecase':  'surfactant adsorption'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'lmp_minimization.input',
    identifier='lmp_minimization.input',
    metadata={
        'type':     'input',
        'language': 'LAMMPS',
        'usecase':  'minimization'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'lmp_equilibration_nvt.input',
    identifier='lmp_equilibration_nvt.input',
    metadata={
        'type':     'input',
        'language': 'LAMMPS',
        'usecase':  'nvt equilibration'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'lmp_equilibration_npt.input',
    identifier='lmp_equilibration_npt.input',
    metadata={
        'type':     'input',
        'language': 'LAMMPS',
        'usecase':  'npt equilibration'})

jobadmin.fp.add_file(
    template_prefix + os.sep + 'lmp_production.input',
    identifier='lmp_production.input',
    metadata={
        'type':     'input',
        'language': 'LAMMPS',
        'usecase':  'production'})

jobadmin.fp.delete_file(identifier='lmp_production_mixed.input')

jobadmin.fp.add_file(
    template_prefix + os.sep + 'lmp_production_mixed.input',
    identifier='lmp_production_mixed.input',
    metadata={
        'type':     'input',
        'language': 'LAMMPS',
        'usecase':  'production'})

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/dat/work/testuser/git/N_surfactant_on_substrate_template/ipynb/fw/N_surfactant_on_substrate_template/jlh_vmd.tcl'

In [27]:
content, doc = jobadmin.fp.get_file(identifier='lmp_production_mixed.input')

In [140]:
print(content.decode())

############
# PRODUCTION
#
if "${has_indenter} > 0" then &
  "variable pressured_group string nonindenter" &
else &
  "variable pressured_group string all"

if "${has_indenter} >0 && ${constant_indenter_velocity} != 0.0" then &
  "velocity indenter set 0.0 0.0 ${constant_indenter_velocity} sum no units box"
#                        ^vx ^vy ^vy
# The sum keyword is used by all styles, except zero. The new velocities will be
# added to the existing ones if sum = yes, or will replace them if sum = no.
# The units keyword is used by set and ramp. If units = box, the velocities and
# coordinates specified in the velocity command are in the standard units 
# described by the units command (e.g. Angstroms/fmsec for real units). If
# units = lattice, velocities are in units of lattice spacings per time (e.g.
# spacings/fmsec) and coordinates are in lattice spacings. The lattice command
# must have been previously used to define the lattice spacing.

# treats indenter rigid, i.e. removes all f

In [28]:
# pull simulation datafile from db
content, doc = jobadmin.fp.get_file(identifier='surfactant_on_AU_111_df_json')

sim_df = pd.read_json(content, orient='index')

ValueError: Invalid file path or buffer object type: <class 'NoneType'>

In [29]:
len(sim_df)

NameError: name 'sim_df' is not defined

In [41]:
sim_df.loc['1010_CTAB_on_AU_111_63x36x2_bilayer']

box                                        [1.82e-08, 1.8000000000000002e-08, 1.800000000...
ci_initial_placement                                                                  random
counterion                                                                                BR
indenter                                                                                None
indenter_pdb                                                                            None
pbc                                                                                      111
pressure                                                                                   1
sb_area                                                                          3.26671e-16
sb_area_per_sf_molecule                                                            3.233e-19
sb_circular_area_per_sf_molecule_radius                                                2e-10
sb_crystal_plane                                                      

In [42]:
# OR: read simulation data from file
# sim_df = pd.read_pickle(sim_data_prefix + os.sep + 'CTAB_on_AU_111.pkl')

In [28]:
sim_df['sf_preassembly'].unique()

array(['bilayer', 'bilayer_with_counterion', 'cylinders_with_counterion',
       'hemicylinders_with_counterion',
       'inverse_monolayer_with_counterion', 'monolayer_with_counterion',
       'inverse_bilayer_with_counterion'], dtype=object)

In [30]:
jobadmin.use_bwcloud_queue()
jobadmin.use_bwcloud_templates()

In [31]:
jobadmin._sim_df = sim_df

In [32]:
# jobadmin._sim_df = sim_df.loc[
#     (sim_df['sf_preassembly'] == 'cylinders_with_counterion') \
#     | (sim_df['sf_preassembly'] == 'hemicylinders_with_counterion') ]

In [33]:
os.getlogin()

OSError: [Errno 25] Inappropriate ioctl for device

In [34]:
template_geninfo = jobadmin._template_geninfo

template_geninfo

'Generated on {datetime:s} by {user:s}@{machine:s}'

In [35]:
machine = os.uname().nodename

In [36]:
geninfo = template_geninfo.format(
    datetime=datetime.datetime.now().ctime(),
    user='testuser',
    machine=machine)

In [37]:
geninfo

'Generated on Fri Feb 15 13:04:46 2019 by testuser@jlh-cloud-28jan19'

In [38]:
# on NEMO
jobadmin.geninfo(reset=True,static_str=geninfo)

'Generated on Fri Feb 15 13:04:46 2019 by testuser@jlh-cloud-28jan19'

## Reinit

### on NEMO
rerun everytime methods have been added or their def's altered in order to reinstantiate JobAdmin object

In [39]:
prefix

'/mnt/dat/work/testuser/adsorption'

In [40]:
#  NEMO
from fwtools.JobAdmin import JobAdmin
from fwtools.JobAdmin import l_CTAB, head_atom_number_CTAB, tail_atom_number_CTAB
from fwtools.JobAdmin import l_SDS, head_atom_number_SDS, tail_atom_number_SDS

# prefix = '/work/ws/nemo/fr_jh1130-201708-0/jobs/lmplab/sds/201810'
template_prefix = prefix + os.sep + 'N_surfactant_on_substrate_template'
output_prefix = prefix + os.sep + 'sys'

jobadmin = JobAdmin( 
    mongodb_host = 'localhost',
    mongodb_port = 27018,
    mongodb_name = 'fireworks-jhoermann',
    queue   = 'BWCLOUD',
    template_prefix = template_prefix,
    output_prefix   = output_prefix,
    sim_df = sim_df
)
jobadmin.std_worker = 'bwcloud_std'
jobadmin.use_bwcloud_templates()
jobadmin.use_bwcloud_queue()
jobadmin._template_lmp_cmd = ' '.join((
    'module purge;',
    'module load LAMMPS;',
    'mpirun ${{MPIRUN_OPTIONS}} lmp -in {inputFile:s}'))
jobadmin.geninfo(reset=True,static_str=geninfo)

'Generated on Fri Feb 15 13:04:46 2019 by testuser@jlh-cloud-28jan19'

## Pick subselection of systems

In [36]:
# several samples for system selections
# system_name_scope should be the overall scope of systems to look at, i.e. all CTAB systems
# system_names is meant as a subset of system_nbame_scope to be worked on 

In [37]:
preassemblies = {
    'cylinders_with_counterion',
    'hemicylinders_with_counterion',
    'bilayer_with_counterion',
    'monolayer_with_counterion',
    'inverse_monolayer_with_counterion'}

In [38]:
type(preassemblies)

set

In [ ]:
system_name_scope = list( sim_df[ 
    sim_df['sf_preassembly'].isin(preassemblies) ].index
                   )

In [111]:
# original SDS systems without indenter
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'SDS')
    ].index )

In [ ]:
system_name_scope = list( sim_df[ 
    sim_df['sf_preassembly'].isin(preassemblies) ].index
                   )

In [ ]:
# original systems without indenter
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna()
    ].index )

In [39]:
# original CTAB systems without indenter
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'CTAB' )
    ].index )

In [45]:
# original SDS systems without indenter
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'SDS' )
    ].index )

In [41]:
# SDS with indenter
system_name_scope = list( sim_df[ 
    (sim_df['indenter'] == '50Ang_stepped') &
    (sim_df['surfactant'] == 'SDS' ) ].index )

In [ ]:
# CTAB with indenter
system_name_scope = list( sim_df[ 
    (sim_df['indenter'] == '50Ang_stepped') &
    (sim_df['surfactant'] == 'CTAB' ) ].index )

In [44]:
# original SDS systems without indenter, counterions at polar heads
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'SDS') & \
        (sim_df['ci_initial_placement'] == 'at_polar_head') & \
        (sim_df["sb_name"] == 'AU_111_51x30x2') & \
        (sim_df['sf_preassembly'] != 'inverse_monolayer_with_counterion') & \
        (sim_df['sf_preassembly'] != 'inverse_bilayer_with_counterion')
    ].index )

In [ ]:
# original CTAB systems without indenter, counterions at polar heads
system_name_scope = list( sim_df[ 
        sim_df['indenter'].isna() & \
        (sim_df['surfactant'] == 'CTAB') & \
        (sim_df['ci_initial_placement'] == 'at_polar_head') & \
        (sim_df["sb_name"] == 'AU_111_63x36x2') & \
        (sim_df['sf_preassembly'] != 'inverse_monolayer_with_counterion') & \
        (sim_df['sf_preassembly'] != 'inverse_bilayer_with_counterion')
    ].index )

In [46]:
system_name_scope

['103_SDS_on_AU_111_60x35x2_bilayer',
 '103_SDS_on_AU_111_60x35x2_bilayer_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_bilayer',
 '1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_inverse_bilayer_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '123_SDS_on_AU_111_17x10x2_bilayer',
 '123_SDS_on_AU_111_17x10x2_bilayer_with_counterion',
 '129_SDS_on_AU_111_51x30x2_bilayer',
 '129_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
 '129_SDS_on_AU_111_51x30x2_inverse_bilayer_with_counterion',
 '129_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion',
 '129_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '14_SDS_on_AU_111_17x10x2_bi

# Initial systems (no indenter)

## Prepare substrates

if necessary

In [45]:
# TODO: change to "wrapper" & batch_prepare_fw stlye, see other fireworks below

In [46]:
prep_subs_wf_dict = jobadmin.prepare_substrates(system_name_scope)

In [96]:
prep_subs_wf_dict['AU_111_17x10x2'][0].as_dict()

{'created_on': '2019-01-27T20:42:33.266783',
 'fw_id': -25,
 'name': 'AU_111_17x10x2_sb_replicate',
 'spec': {'_category': 'bwcloud_std',
  '_dupefinder': {'_fw_name': 'DupeFinderExact'},
  '_tasks': [{'_fw_name': 'FileTransferTask',
    'files': [{'dest': 'au_cell_P1_111.gro',
      'src': '/mnt/dat/work/testuser/adsorption/N_surfactant_on_substrate_template/au_cell_P1_111.gro'}],
    'mode': 'copy'},
   {'_fw_name': 'ScriptTask',
    'fizzle_bad_rc': True,
    'script': ['module load MDTools; module load GROMACS; replicate.sh 17 10 2 111'],
    'stderr_file': 'AU_111_17x10x2_sb_replicate.err',
    'stdout_file': 'AU_111_17x10x2_sb_replicate.out',
    'use_shell': True},
   {'_fw_name': 'DeleteFilesTask', 'identifiers': ['AU_111_17x10x2.pdb']},
   {'_fw_name': 'AddFilesTask',
    'identifiers': ['AU_111_17x10x2.pdb'],
    'paths': ['AU_111_17x10x2_tidy.pdb']}],
  'geninfo': 'Generated on Sun Jan 27 19:14:50 2019 by testuser@jlh-cloud-10jan19',
  'step': 'sb_replicate',
  'substrate': 

In [97]:
len(prep_subs_wf_dict)

4

In [98]:
prep_subs_fw_id_dict = jobadmin.add_fw(prep_subs_wf_dict)

2019-01-27 20:42:34,723 INFO Added a workflow. id_map: {-25: 1}


INFO:launchpad:Added a workflow. id_map: {-25: 1}


2019-01-27 20:42:34,732 INFO Added a workflow. id_map: {-26: 2}


INFO:launchpad:Added a workflow. id_map: {-26: 2}


2019-01-27 20:42:34,742 INFO Added a workflow. id_map: {-27: 3}


INFO:launchpad:Added a workflow. id_map: {-27: 3}


2019-01-27 20:42:34,750 INFO Added a workflow. id_map: {-28: 4}


INFO:launchpad:Added a workflow. id_map: {-28: 4}


## Prepare aggregates

In [ ]:
# TODO: switch monolayers and bilayers to wrapper & batch_prepare_fw 

In [47]:
system_name_scope

['1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
 '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '129_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
 '129_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '220_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
 '220_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
 '2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '377_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '377_SDS_on_AU_111_51x30x2_cylind

### (hemi-) cylinders

In [ ]:
redo_hemicylinders = [
    '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion']

In [ ]:
redo_cylinders = [
    '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion' ]

In [61]:
redo_cylinders = [ "1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion" ]

In [54]:
jobadmin.packmol_queueadapter

{}

In [ ]:
jobadmin.packmol_queueadapter["walltime"] = '96:00:00'

In [115]:
# SDS cylinders
pack_cylinders_wrapper = lambda sfN, sb_measures, surfactant, counterion: \
    jobadmin.pack_cylinders(sfN, sb_measures, surfactant, counterion,
        hemicylinders=False, l_surfactant=l_SDS, 
        head_atom_number=head_atom_number_SDS,
        tail_atom_number=tail_atom_number_SDS)

In [116]:
# SDS hemicylinders
pack_cylinders_wrapper = lambda sfN, sb_measures, surfactant, counterion: \
    jobadmin.pack_cylinders(sfN, sb_measures, surfactant, counterion,
        hemicylinders=True, l_surfactant=l_SDS, 
        head_atom_number=head_atom_number_SDS,
        tail_atom_number=tail_atom_number_SDS)

In [117]:
prepare_packmol_wrapper = lambda system_name: jobadmin.prepare_packmol(
    system_name, pack_cylinders_wrapper, nloop=20, maxit=50)

In [64]:
system_names = redo_cylinders

In [113]:
system_names = [ '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion' ]

In [114]:
system_names

['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion']

In [118]:
fw_dict = jobadmin.batch_prepare_fw( system_names, prepare_packmol_wrapper )

In [119]:
jobadmin.add_wf(fw_dict)

2019-01-28 10:02:47,457 INFO Added a workflow. id_map: {-38: 13, -37: 14}


INFO:launchpad:Added a workflow. id_map: {-38: 13, -37: 14}


{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': {-38: 13, -37: 14}}

### (inverse) monolayer

In [ ]:
redo_monolayer = ['2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion']

In [337]:
redo_monolayer = ["1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion"]

In [338]:
pack_monolayer_wrapper = lambda sfN, sb_measures, surfactant, counterion: \
    jobadmin.pack_monolayer(sfN, sb_measures, surfactant, counterion,
        l_surfactant=l_SDS, 
        head_atom_number=head_atom_number_SDS,
        tail_atom_number=tail_atom_number_SDS)

In [339]:
prepare_packmol_wrapper = lambda system_name: jobadmin.prepare_packmol(
    system_name, pack_monolayer_wrapper, nloop=200, maxit=500)

In [340]:
system_names = redo_monolayer

In [341]:
fw_dict = jobadmin.batch_prepare_fw( system_names, prepare_packmol_wrapper )

In [342]:
jobadmin.add_wf(fw_dict)

2018-10-19 21:19:43,258 INFO Added a workflow. id_map: {-195: 10695, -194: 10696}


INFO:launchpad:Added a workflow. id_map: {-195: 10695, -194: 10696}


{'1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion': {-195: 10695,
  -194: 10696}}

### bilayer

In [ ]:
redo_bilayer = ['2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion' ]

In [ ]:
pack_bilayer_wrapper = lambda sfN, sb_measures, surfactant, counterion: \
    jobadmin.pack_bilayer(sfN, sb_measures, surfactant, counterion,
        l_surfactant=l_SDS, 
        head_atom_number=head_atom_number_SDS,
        tail_atom_number=tail_atom_number_SDS)

In [ ]:
prepare_packmol_wrapper = lambda system_name: jobadmin.prepare_packmol(
    system_name, pack_bilayer_wrapper, nloop=200, maxit=500)

In [ ]:
system_names = redo_bilayer

In [ ]:
fw_dict = jobadmin.batch_prepare_fw( system_names, prepare_packmol_wrapper )

In [ ]:
jobadmin.add_wf(fw_dict)

In [ ]:
logging.info("Querying appended packing workflows.")
packmol_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='packmol',
    state=["FIZZLED","COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

# fizzled might be ok if walltime expired, 
# packmol gets good results half-way

logging.info("Preparing LAMMPS conversion")
pdb2lmp_wf_dict = jobadmin.prepare_pdb2lmp(system_names)
logging.info("Appening LAMMPS conversion")
jobadmin.append_wf_by_key(pdb2lmp_wf_dict, packmol_fw_id_dict)

logging.info("Querying appended LAMMPS conversion workflows")
ch2lmp_fw_id_dict = jobadmin.query_step(step='ch2lmp')

### finish solvation & format conversions

In [ ]:
system_names = redo

In [368]:
being_redone = {
    '1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
    '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
    '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
    '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion'}

In [369]:
system_names = being_redone

In [343]:
system_names = redo_monolayer

In [142]:
system_names

['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion']

In [136]:
packmol_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='packmol',
    state=["FIZZLED","COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])


In [148]:
packmol_fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [13]}

In [159]:
pdb2lmp_wf_dict = jobadmin.batch_prepare_fw(system_names, jobadmin.prepare_pdb2lmp)

In [160]:
pdb2lmp_wf_dict['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion'].as_dict()

{'created_on': datetime.datetime(2019, 1, 28, 13, 0, 33, 431230),
 'fws': [{'created_on': '2019-01-28T13:00:33.430512',
   'fw_id': -69,
   'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_recover_packmol',
   'spec': {'_allow_fizzled_parents': True,
    '_category': 'bwcloud_std',
    '_dupefinder': {'_fw_name': 'DupeFinderExact'},
    '_files_in': {'packmol_pdb': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_packmol.pdb',
     'packmol_pdb_FORCED': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_packmol.pdb_FORCED'},
    '_files_out': {'packmol_pdb': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_packmol.pdb'},
    '_tasks': [{'_fw_name': 'ScriptTask',
      'fizzle_bad_rc': False,
      'script': ['mkdir -p "/mnt/dat/work/testuser/adsorption/sys/646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion"'],
      'use_shell': True},
     {'_fw_name': 'RecoverPackmolTask',
      'dest': '/mnt/dat/work/testuser/adsorption/sys/646_SDS_on_AU_11

In [161]:
logging.info("Appening LAMMPS conversion")
jobadmin.append_wf_by_key(pdb2lmp_wf_dict, packmol_fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [13]


In [125]:
logging.info("Querying appended LAMMPS conversion workflows")
ch2lmp_fw_id_dict = jobadmin.query_step(step='ch2lmp')

In [126]:
ch2lmp_fw_id_dict

{'1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [7],
 '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [15]}

## Query

In [41]:
system_name_scope = [ 
     '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion' ]

In [163]:
healthy_prepared_systems = jobadmin.get_set_of_prepared_minimizations()

In [164]:
# check where subsequent steps are missing

In [165]:
to_initialize_formats = jobadmin.identifyIncompleteWorkflowsInScope(
    system_name_scope=system_name_scope, precedent_step_name='packmol', subsequent_step_name='ch2lmp')

In [166]:
to_initialize_formats

{}

In [177]:
# Use ch2lmp as reference
to_minimize = jobadmin.identifyIncompleteWorkflowsInScope(precedent_step_name='ch2lmp',
    subsequent_step_name='minimization', system_name_scope=system_name_scope)

In [178]:
to_minimize

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [39]}

In [173]:
jobadmin.defuse_children(19,dry_run=True)

[15, 16, 17, 18, 19]

In [176]:
jobadmin.defuse_children(33)

[31, 32, 33]

In [ ]:
to_minimize = jobadmin.identifyIncompleteWorkflowsInScope(precedent_step_name='prepare_system_files',
    subsequent_step_name='minimization', system_name_scope=system_name_scope)

In [ ]:
to_minimize

In [ ]:
redo = (set(system_name_scope) - healthy_prepared_systems) | set(to_minimize.keys())

In [ ]:
redo

In [387]:
to_equilibrate_nvt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='minimization', subsequent_step_name='equilibration_nvt',
    system_name_scope=system_name_scope)

In [1]:
to_equilibrate_nvt

NameError: name 'to_equilibrate_nvt' is not defined

In [ ]:
to_equilibrate_npt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_nvt',
    subsequent_step_name='equilibration_npt',
    system_name_scope=system_name_scope)

In [ ]:
to_equilibrate_npt

In [ ]:
to_run_production = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_npt',
    subsequent_step_name='10ns_production_mixed',
    system_name_scope=system_name_scope)

In [ ]:
to_run_production

In [ ]:
to_run_production = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_npt',
    subsequent_step_name='production',
    system_name_scope=system_name_scope)

In [ ]:
to_run_production

In [ ]:
unfinished_systems = to_run_production.keys()

In [ ]:
previous_runs = jobadmin.query_systems(unfinished_systems,step='10ns_production_mixed')

In [316]:
discontinued_prodcution_runs = jobadmin.get_discontinued_production_runs()

In [ ]:
completed_productions_dict

## LAMMPS jobs

In [ ]:
# TODO: switch to wrapper & batch_prepare_fw style

In [ ]:
# Template for appending LAMMPS runs
prep_fw_dict = jobadmin.prepare_systems(system_names)
jobadmin.append_wf_by_key( prep_fw_dict, ch2lmp_fw_id_dict )
prep_fw_id_dict = jobadmin.query_step(
    step = 'prepare_system_files')

In [ ]:
# TODO: update "prepare_restarts" to enable restarts with colvars module
prod_fw_dict = jobadmin.prepare_production(system_names)
jobadmin.append_wf_by_key(prod_fw_dict, npt_fw_id_dict)
prod_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='10ns_production_mixed',
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

restart_fw_dict = jobadmin.prepare_restarts(system_names)
jobadmin.append_wf_by_key(restart_fw_dict, prod_fw_id_dict)

### Preparation

In [189]:
fw_id_dict = jobadmin.query_systems(
    system_names,step="ch2lmp",
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [190]:
fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [39]}

In [ ]:
# Certain steps push representative data files to the database.

In [191]:
pull_datafile_from_db_wrapper = lambda system_name: \
    jobadmin.pull_datafile_from_db(system_name, data_file_identifier_suffix='_psfgen.data')

In [192]:
fw_dict = jobadmin.batch_prepare_fw( system_names, pull_datafile_from_db_wrapper )

In [193]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [39]


### Minimization

In [357]:
jobadmin.minimization_queueadapter

{'nodes': 1, 'walltime': '24:00:00'}

In [358]:
jobadmin.minimization_queueadapter["nodes"] = 2

In [ ]:
jobadmin.minimization_queueadapter["walltime"] = '24:00:00'

In [194]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='pull_datafile_from_db', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [195]:
fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [47]}

In [182]:
jobadmin._template_lmp_cmd = 'module load LAMMPS; mpirun -n 4 lmp -in {inputFile:s}' # for testing on bwcloud

In [183]:
jobadmin._template_lmp_cmd

'module load LAMMPS; mpirun -n 4 lmp -in {inputFile:s}'

In [196]:
minimize_wrapper = lambda system_name: jobadmin.minimize(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} -v has_indenter 0',
        '-v robust_minimization 1 -v pbc2d 0 -v surfactant_name SDS',
        '-v mpiio 0 -v freeze_substrate 1')))

In [197]:
fw_dict = jobadmin.batch_prepare_fw( system_names, minimize_wrapper )

In [198]:
fw_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': {"spec": {"_queueadapter": {}, "_category": "bwcloud_std", "_dupefinder": {"_fw_name": "DupeFinderExact"}, "_files_in": {"data_file": "datafile.lammps"}, "_files_out": {"data_file": "646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_minimized.lammps"}, "system_name": "646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion", "step": "minimization", "geninfo": "Generated on Sun Jan 27 19:14:50 2019 by testuser@jlh-cloud-10jan19", "_tasks": [{"files": [{"src": "/mnt/dat/work/testuser/adsorption/N_surfactant_on_substrate_template/lmp_header.input", "dest": "lmp_header.input"}, {"src": "/mnt/dat/work/testuser/adsorption/N_surfactant_on_substrate_template/lmp_minimization.input", "dest": "lmp_minimization.input"}], "mode": "copy", "_fw_name": "FileTransferTask"}, {"stdout_file": "lmp_minimization.out", "stderr_file": "lmp_minimization.err", "use_shell": true, "fizzle_bad_rc": true, "script": ["module load LAMMPS; mpirun -n 4 

In [199]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [47]


### NVT equilibration

In [379]:
jobadmin.equilibration_queueadapter

{'nodes': 4, 'walltime': '24:00:00'}

In [ ]:
jobadmin.equilibration_queueadapter["nodes"] = 4

In [364]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='minimization', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

# better alterbative:

In [387]:
to_equilibrate_nvt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='minimization', subsequent_step_name='equilibration_nvt',
    system_name_scope=system_name_scope)

In [388]:
to_equilibrate_nvt

{'1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10691],
 '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10644],
 '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [10704],
 '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10645],
 '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10638],
 '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10634],
 '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10636],
 '2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion': [10642],
 '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10640],
 '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10641],
 '2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [10639],
 '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10635],
 '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10643],
 '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10637]}

In [389]:
fw_id_dict = to_equilibrate_nvt

In [390]:
nvtEquilibrate_wrapper = lambda system_name: jobadmin.nvtEquilibrate(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s}', 
        '-v has_indenter 0 -v pbd2d 0 -v reinitialize_velocities 1',
        '-v surfactant_name SDS -v mpiio 0')))

In [391]:
fw_dict = jobadmin.batch_prepare_fw( system_names, nvtEquilibrate_wrapper )

In [392]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 220_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10634]
Append 1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10691]
Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10704]
Append 220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10636]
Append 2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10640]
Append 129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10638]
Append 2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10641]
Append 2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10639]
Append 377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10635]
Append 2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion to last fw_id in list [10642]
Append 75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10637

### NPT equilibration

In [ ]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='equilibration_nvt', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

# better alternative:

In [393]:
to_equilibrate_npt = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_nvt',
    subsequent_step_name='equilibration_npt',
    system_name_scope=system_name_scope)

In [397]:
fw_id_dict = to_equilibrate_npt

In [398]:
nptEquilibrate_wrapper = lambda system_name: jobadmin.nptEquilibrate(
    system_name,
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s}',
        '-v dataFile {dataFile:s}',
        '-v has_indenter 0',
        '-v pbd2d 0 -v reinitialize_velocities 0',
        '-v surfactant_name SDS')) )

In [399]:
fw_dict = jobadmin.batch_prepare_fw( system_names, nptEquilibrate_wrapper )

In [400]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 220_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10705]
Append 1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10706]
Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10707]
Append 220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10708]
Append 2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10709]
Append 129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10710]
Append 2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10711]
Append 2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10712]
Append 377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10713]
Append 2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion to last fw_id in list [10714]
Append 75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10715

### Production

In [401]:
jobadmin.production_queueadapter

{'nodes': 8, 'walltime': '96:00:00'}

In [ ]:
jobadmin.production_queueadapter["nodes"] = 8

In [ ]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='equilibration_npt', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [407]:
to_run_production = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='equilibration_npt',
    subsequent_step_name='production',
    system_name_scope=system_names)

In [410]:
fw_id_dict = to_run_production

In [411]:
production_wrapper = lambda system_name: jobadmin.production(
    system_name,
    total_steps = 5000000, 
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s}',
        '-v dataFile {dataFile:s}',
        '-v has_indenter 0',
        '-v pbd2d 0', # no 2d-periodic, but 3d-periodic BC
        '-v reinitialize_velocities 0',
        '-v use_colvars 0',
        '-v productionSteps 5000000', # 5000000 ~ 10 ns
        '-v mpiio 0', # does not work yet with current built, recompile
        '-v surfactant_name SDS')) )

In [412]:
fw_dict = jobadmin.batch_prepare_fw( system_names, production_wrapper )

In [413]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 220_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10719]
Append 1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10720, 7650]
Append 1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [7234, 10721]
Append 220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10722]
Append 2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion to last fw_id in list [10723]
Append 129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10724]
Append 2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10725]
Append 2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion to last fw_id in list [10726]
Append 377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in list [10727]
Append 2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion to last fw_id in list [10728]
Append 75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion to last fw_id in

## Check progress and (re-) run if necessary

In [ ]:
# REWORK functionality here, outdated

In [ ]:
healthy_production_runs = jobadmin.get_set_of_healthy_production_runs()

In [ ]:
completed_production_runs = jobadmin.get_set_of_evolved_systems()

In [ ]:
system_names = list(completed_production_runs & set(system_name_scope))

In [ ]:
system_names

In [ ]:
incomplete_but_healthy_production_runs = healthy_production_runs - completed_production_runs

In [ ]:
incomplete_but_healthy_production_runs

In [ ]:
discontinued_production_runs = jobadmin.get_discontinued_production_runs()

In [ ]:
discontinued_production_runs

In [ ]:
system_name_scope = system_names

In [ ]:
to_prepare = jobadmin.get_to_prepare()

In [ ]:
system_names = list( set(system_name_scope) & set(to_prepare) )

In [ ]:
to_minimize = jobadmin.get_to_minimize()

In [ ]:
system_names = list( set(system_name_scope) & set(to_minimize) )

In [ ]:
system_names

### NVT eq

In [ ]:
to_nvt_equilibrate = jobadmin.get_to_nvt_equlibrate()

In [ ]:
to_nvt_equilibrate

In [ ]:
system_names = list(set(to_nvt_equilibrate) & set(system_name_scope))

In [ ]:
system_names

### NPT eq

In [ ]:
to_npt_equilibrate = jobadmin.get_to_npt_equlibrate()

In [ ]:
system_names = list(set(to_npt_equilibrate) & set(system_name_scope))

In [ ]:
system_names

### Production

In [ ]:
to_run_production = jobadmin.get_to_run_production()

In [ ]:
system_names = list(set(to_run_production) & set(system_name_scope))

In [ ]:
# manually delete a file from Filepad db
jobadmin.fp.delete_file(
    identifier='1010_CTAB_on_AU_111_63x36x2_cylinders_with_counterion_psfgen.data')

## Recovering stuff

### Defuse dead ends & children

In [ ]:
steps = ['prepare_system_files',
         'minimization', 
         'equilibration_nvt', 
         'equilibration_npt',
        ]

In [ ]:
step_fizzled_dict = {}
for step in steps:
    step_fizzled_dict[step] = jobadmin.query_step(
        step = step, state_list = ["FIZZLED"] )

In [ ]:
# only defuse systems in scope
reduced_step_fizzled_dict = {}
for step, fw_id_dict in step_fizzled_dict.items():
    reduced_step_fizzled_dict[step] = { system_name: fw_id_lst \
        for system_name, fw_id_lst in fw_id_dict.items() \
        if system_name in system_name_scope }

In [ ]:
reduced_step_fizzled_dict

In [ ]:
defused_fw_id_list = []
for step, fw_id_dict in reduced_step_fizzled_dict.items():
    for system_name, fw_id_lst in fw_id_dict.items():
        for fw_id in fw_id_lst:
            logging.info("Defuse {} : {} : {} and its children...".format(
                step, system_name, fw_id ) )
            defused_fw_id_list.extend( jobadmin.defuse_children(fw_id) )
        

### Defuse misqueued step (and subsequent ones)

In [374]:
fw_id_dict = jobadmin.query_systems(system_names,step="equilibration_nvt", state=["WAITING"])

In [375]:
fw_id_dict

{'1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10692],
 '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10656],
 '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [],
 '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10657],
 '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10650],
 '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10646],
 '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10648],
 '2214_SDS_on_AU_111_51x30x2_bilayer_with_counterion': [10654],
 '2214_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10652],
 '2214_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10653],
 '2214_SDS_on_AU_111_51x30x2_monolayer_with_counterion': [10651],
 '377_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10647],
 '75_SDS_on_AU_111_51x30x2_cylinders_with_counterion': [10655],
 '75_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion': [10649]}

In [376]:
defused_fw_id_list = []
for system_name, fw_id_lst in fw_id_dict.items():
    for fw_id in fw_id_lst:
        logging.info("Defuse {} : {} : {} and its children...".format(
            step, system_name, fw_id ) )
        defused_fw_id_list.extend( jobadmin.defuse_children(fw_id) )

### Recover packmol runs
packmol runs can fail due to walltime. However, often the (not yet converged) configurations are fine to use.

In [ ]:
packmol_fw_id_dict = jobadmin.query_step(step='packmol')

In [ ]:
packmol_fw_id = packmol_fw_id_dict['1298_CTAB_on_AU_111_63x36x2_cylinders_with_counterion'][-1]

In [ ]:
packmol_fw_id

In [ ]:
packmol_launchdir = jobadmin.lpad.get_launchdir(packmol_fw_id)

In [ ]:
system_name = '1298_CTAB_on_AU_111_63x36x2_cylinders_with_counterion'

In [ ]:
# for systems with "fizzled" packmol tasks, append "dummy" restart parents
# They run in the same directory as previous fizzled packmol and forward the latest configurations
dummy_parent_fw_dict = {}
for system_name in system_names:
    packmol_launchdir = jobadmin.lpad.get_launchdir( 
        packmol_fw_id_dict[system_name][-1] )
    dummy_parent_fw = Firework( [
            DummyParentTask() 
        ],
        spec = {
            '_category':   jobadmin.std_worker,
            '_launch_dir': packmol_launchdir,
            '_files_out': {
                'packmol_pdb': system_name + '_packmol.pdb',
                'packmol_pdb_FORCED': system_name + '_packmol.pdb_FORCED'
            },
            'system_name': system_name,
            'geninfo':     jobadmin.geninfo(),
            'step':        'packmol_dummy'
        },
        name = system_name + '_dummy_parent'
    )
    dummy_parent_fw_dict[system_name] = dummy_parent_fw

In [ ]:
jobadmin.append_wf_by_key(dummy_parent_fw_dict, packmol_fw_id_dict)

In [ ]:
packmol_dummy_fw_id_dict = jobadmin.query_step('packmol_dummy')

In [ ]:
packmol_dummy_fw_id_dict

In [ ]:
system_names

In [ ]:
# delete previously written packmol results from db 
# (otherwise, will not be overwritten by recovered packmol results)
for system_name in system_names:
    file_id = system_name + '_psfgen.data'
    jobadmin.fp.delete_file(file_id)

In [ ]:
# sample for a single packmol recovery task
recover_packmol_fw = Firework( 
    RecoverPackmolTask( { 
        'dest':   output_prefix + os.sep + system_name,
        'recover': True,
        'glob_patterns': [ '*_restart.pack',
            '*_packmol.pdb', '*_packmol.pdb_FORCED', '*_packmol.inp'],
        'forward_glob_patterns': { 
            "packmol_pdb" : ["*_packmol.pdb_FORCED", "*_packmol.pdb"] }
    } ),
    spec = {
        '_category':   jobadmin.std_worker,
        '_files_in': {
            'packmol_pdb': system_name + '_packmol.pdb',
            'packmol_pdb_FORCED': system_name + '_packmol.pdb_FORCED'
        },
        'system_name': system_name,
        'step':        'recover_packmol',
        'geninfo':     jobadmin.geninfo()
    },
    name = system_name + '_recover_packmol'
)

In [ ]:
jobadmin.lpad.append_wf( Workflow( [ recover_packmol_fw ] ), [8976] )

## Append data file strorage
after successfull production, insert final configurationms into database

In [ ]:
# first, identify files not yet stored

In [320]:
# completed_productions_dict = jobadmin.query_systems(
#     system_name_scope, step="10ns_production_mixed", state=["COMPLETED"])

In [414]:
completed_productions_dict = jobadmin.query_step(step="10ns_production_mixed", state_list=["COMPLETED"])

In [418]:
completed_store_data_file_dict = jobadmin.query_step(step="store_data_file", state_list=["COMPLETED"])

In [423]:
# only check for current system_name_scope
system_names = list( 
    ( set(completed_10ns_productions_dict.keys()) - 
         set(completed_store_data_file_dict.keys() ) ) & set(system_name_scope)
)

# or for all systems
# system_names = list(set(system_names) & set(system_name_scope))

In [415]:
uncompleted_productions_in_scope = set(system_name_scope) - set(completed_productions_dict.keys())

In [ ]:
fw_id_dict = jobadmin.query_systems(system_names, step='10ns_production_mixed', state='COMPLETED')

In [ ]:
fw_id_dict

In [ ]:
fw_dict = jobadmin.batch_prepare_fw( system_names, jobadmin.store_data_file )

In [ ]:
len(fw_dict)

In [ ]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

In [ ]:
# identify fizzled fireworks and rerun

In [ ]:
fizzled_fw_id_dict = jobadmin.query_step(
    step='store_data_file', 
    state_list=["FIZZLED"])

In [ ]:
fizzled_fw_id_dict

In [ ]:
for key, fw_id_lst in fizzled_fw_id_dict.items():
    jobadmin.lpad.rerun_fw(fw_id_lst[-1], rerun_duplicates=False)

In [ ]:
fizzled_fw_id_dict = jobadmin.query_systems(
    system_names, 
    step='store_data_file', 
    state=["FIZZLED"])

# Indenter systems


## Query finished interface productions

In [ ]:
# # original SDS systems without indenter
# system_name_scope = list( sim_df[ 
#         sim_df['indenter'].isna() & \
#         (sim_df['surfactant'] == 'SDS')
#     ].index )

In [52]:
fw_id_dict = jobadmin.query_step(
    step='store_data_file', 
    state_list=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [53]:
len(fw_id_dict)

50

In [54]:
system_names = set(system_name_scope) & set(fw_id_dict.keys())

In [55]:
len(system_names)

13

In [56]:
system_names

{'1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '129_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '220_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '220_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '377_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '377_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '377_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '646_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '646_SDS_on_AU_111_51x30x2_cylinders_with_counterion',
 '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion',
 '646_SDS_on_AU_111_51x30x2_monolayer_with_counterion',
 '75_SDS_on_AU_111_51x30x2_bilayer_with_counterion',
 '75_SDS_on_AU_111_51x30x2_monolayer_with_counterion'}

In [59]:
jobadmin.fp.count()

221

In [58]:
jobadmin.fp.count(filter={'identifier': {'$regex': '.*SDS.*'}})

113

In [69]:
# Display files stored in filepad
for p in jobadmin.fp.filepad.find( { 'identifier': {'$regex': '.*SDS_on_AU_111_51x30x2_.*_10ns\.lammps'}}):
    print(p['identifier'])

1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_10ns.lammps
1107_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_10ns.lammps
129_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_10ns.lammps
129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_10ns.lammps
220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_10ns.lammps
220_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_10ns.lammps
220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_10ns.lammps
377_SDS_on_AU_111_51x30x2_bilayer_with_counterion_10ns.lammps
377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_10ns.lammps
377_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_10ns.lammps
377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_10ns.lammps
646_SDS_on_AU_111_51x30x2_bilayer_with_counterion_10ns.lammps
646_SDS_on_AU_111_51x30x2_cylinders_with_counterion_10ns.lammps
646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_10ns.lammps
646_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_10ns.lammps
64

## Workflow build up

### Initiate indenter workflows

In [42]:
interface_file_suffix = '_10ns.lammps'
indenter = '50Ang_stepped'
indenter_file = '50Ang_stepped.pdb'

In [43]:
system_names = ['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion']

In [44]:
len(system_names)

1

In [45]:
# construct names for new systems containing indenter
indenter_system_names_dict = {}
for system_name in system_names:
    indenter_system_names_dict[ system_name + '_' + indenter ] = system_name 

In [46]:
# SDS systems with indenter
system_name_scope = list( sim_df[ 
        (sim_df['indenter'] ==  indenter) & \
        (sim_df['surfactant'] == 'SDS')
    ].index )

In [47]:
system_name_scope

['1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '1107_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped',
 '1107_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped',
 '1107_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_50Ang_stepped',
 '1107_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped',
 '129_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '129_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped',
 '129_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped',
 '129_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_50Ang_stepped',
 '129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped',
 '220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped',
 '220_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped',
 '220_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped',
 '220_SDS_on_AU_111_51x30x2_inverse_monolayer_with_counterion_5

In [48]:
system_names = set(indenter_system_names_dict.keys()) & set(system_name_scope)

In [49]:
system_names

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'}

In [51]:
len(system_names)

1

In [52]:
jobadmin.geninfo(reset=True,static_str=geninfo)

'Generated on Mon Feb  4 15:55:11 2019 by testuser@jlh-cloud-28jan19'

In [80]:
for system_name in system_names:
    interface_file = indenter_system_names_dict[system_name] + interface_file_suffix
    fw = jobadmin.initiate_indenter_workflow(
        system_name, 
        interface_file = interface_file, 
        indenter_file  = indenter_file )
    jobadmin.lpad.add_wf( Workflow( [ fw ], name = system_name ) )

2019-02-03 13:55:03,309 INFO Added a workflow. id_map: {-2: 11312}


INFO:launchpad:Added a workflow. id_map: {-2: 11312}


In [81]:
# convert is used to turn tga into png (not crucial)
jobadmin._template_convert_cmd

'convert "{infile:s}" "{outfile:s}"'

### Indenter insertion

In [53]:
system_names

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'}

In [83]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='initiate_indenter_workflow', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [84]:
to_insert_indenter = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='initiate_indenter_workflow',
    subsequent_step_name='indenter_insertion',
    system_name_scope=system_name_scope)

In [93]:
to_insert_indenter

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [11312]}

In [94]:
fw_id_dict = to_insert_indenter

In [102]:
indenter_insertion_wrapper = lambda system_name: \
    jobadmin.indenter_insertion(
        system_name, fworker='bwcloud_noqueue', desired_distance=75.0 ) # Ang

In [103]:
fw_dict = jobadmin.batch_prepare_fw( system_names, indenter_insertion_wrapper )

In [104]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [11312]


### Datafile merge

In [108]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='indenter_insertion', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [109]:
to_merge_datafiles = jobadmin.identifyIncompleteWorkflowsInScope(
    precedent_step_name='indenter_insertion',
    subsequent_step_name='pizzapy_merge',
    system_name_scope=system_name_scope)

In [110]:
to_merge_datafiles

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [11313]}

In [111]:
fw_id_dict = to_merge_datafiles

In [112]:
jobadmin._template_pizzapy_merge_cmd

'module load MDTools/jlh-25Jan19-python-2.7; pizza.py -f merge.py "{datafile:s}" "{reffile:s}" "{outfile:s}"'

In [113]:
merge_datafile_wrapper = lambda system_name: \
    jobadmin.add_parameters_to_datafile_and_store(
        system_name, indenter_suffix = '_50Ang_stepped' )

In [114]:
fw_dict = jobadmin.batch_prepare_fw(system_names, 
                                    merge_datafile_wrapper)

In [115]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [11313]


In [199]:
'\u2019'.encode()


b'\xe2\x80\x99'

### Pull file from DB (migrate machine)
this step is only necessary if changeing file system for subsequent steps

In [116]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='pizzapy_merge', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [130]:
pull_datafile_wrapper = lambda system_name: \
    jobadmin.pull_datafile_from_db(
        system_name, fworker='nemo_noqueue' )

In [131]:
fw_dict = jobadmin.batch_prepare_fw( system_names, pull_datafile_wrapper )

In [132]:
fw_dict['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'].as_dict()

{'created_on': '2019-02-03T15:16:46.957956',
 'fw_id': -8,
 'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_pull_datafile_from_db',
 'spec': {'_category': 'nemo_noqueue',
  '_files_out': {'data_file': 'datafile.lammps'},
  '_tasks': [{'_fw_name': 'GetFilesTask',
    'identifiers': ['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_initial.lammps'],
    'new_file_names': ['datafile.lammps']}],
  'geninfo': 'Generated on Sun Feb  3 13:06:46 2019 by testuser@jlh-cloud-28jan19',
  'step': 'pull_datafile_from_db',
  'system_name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'},
 'updated_on': '2019-02-03T15:16:46.957960'}

In [134]:
fw_dict['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped']

{"spec": {"_category": "nemo_noqueue", "_files_out": {"data_file": "datafile.lammps"}, "system_name": "646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped", "step": "pull_datafile_from_db", "geninfo": "Generated on Sun Feb  3 13:06:46 2019 by testuser@jlh-cloud-28jan19", "_tasks": [{"identifiers": ["646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_initial.lammps"], "new_file_names": ["datafile.lammps"], "_fw_name": "GetFilesTask"}]}, "fw_id": -8, "created_on": "2019-02-03T15:16:46.957956", "updated_on": "2019-02-03T15:16:46.957960", "name": "646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_pull_datafile_from_db"}

In [135]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [9891, 9917, 10824, 11314]


### Minimization

In [142]:
jobadmin._template_lmp_cmd

'module purge; module load LAMMPS; mpirun ${{MPIRUN_OPTIONS}} lmp -in {inputFile:s}'

In [166]:
jobadmin.use_nemo_templates()

In [167]:
jobadmin._template_lmp_cmd

'module purge; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-03Feb19; mpirun ${{MPIRUN_OPTIONS}} lmp -in {inputFile:s}'

In [147]:
jobadmin.use_nemo_queue()

In [72]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='pull_datafile_from_db', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [73]:
fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': []}

In [70]:
fw_id_dict = jobadmin.identifyIncompleteWorkflowsInScope(
     precedent_step_name='pull_datafile_from_db',
     subsequent_step_name='minimization',
     system_name_scope=system_name_scope)

In [71]:
fw_id_dict

{}

In [183]:
jobadmin.minimization_queueadapter['queue'] = 'express'

In [201]:
jobadmin.minimization_queueadapter['queue'] = ''

In [207]:
jobadmin.minimization_queueadapter['walltime'] = '08:00:00'

In [203]:
jobadmin.minimization_queueadapter['nodes'] = 2

In [204]:
jobadmin.minimization_queueadapter['ppn'] = 20

In [208]:
jobadmin.minimization_queueadapter

{'nodes': 2, 'ppn': 20, 'queue': '', 'walltime': '08:00:00'}

In [209]:
minimize_wrapper = lambda system_name: jobadmin.minimize(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} -v has_indenter 1 -v mpiio 0',
        '-v robust_minimization 0 -v pbc2d 0 -v compute_interactions 0 -v store_forces 0',
        '-v freeze_substrate 1 -v surfactant_name SDS')))

In [210]:
fw_dict = jobadmin.batch_prepare_fw( system_names, minimize_wrapper )

In [211]:
fw_dict['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'].as_dict()

{'created_on': '2019-02-03T23:40:39.208492',
 'fw_id': -13,
 'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_minimzation',
 'spec': {'_category': 'nemo_queue',
  '_files_in': {'data_file': 'datafile.lammps'},
  '_files_out': {'data_file': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_minimized.lammps'},
  '_queueadapter': {'nodes': 2,
   'ppn': 20,
   'queue': '',
   'walltime': '08:00:00'},
  '_tasks': [{'_fw_name': 'GetFilesTask',
    'identifiers': ['lmp_header.input',
     'lmp_minimization.input',
     'extract_thermo.sh']},
   {'_fw_name': 'ScriptTask',
    'fizzle_bad_rc': True,
    'script': ['module purge; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-03Feb19; mpirun ${MPIRUN_OPTIONS} lmp -in lmp_minimization.input -v baseName 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped -v dataFile datafile.lammps -v has_indenter 1 -v mpiio 0 -v robust_minimization 0 -v pbc2d 0 -v compute_interactions 0 

In [212]:
jobadmin.append_wf_by_key( fw_dict, fw_id_dict )

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [11315]


### NVT equilibration

In [116]:
jobadmin.use_nemo_templates()

In [117]:
jobadmin.use_nemo_queue()

In [118]:
jobadmin._template_lmp_cmd

'module purge; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-04Feb19; mpirun ${{MPIRUN_OPTIONS}} lmp -in {inputFile:s}'

In [109]:
# test on express queue
jobadmin.equilibration_queueadapter['queue'] = 'express'
jobadmin.equilibration_queueadapter['walltime'] = '00:02:00'
jobadmin.equilibration_queueadapter['nodes'] = 1
jobadmin.equilibration_queueadapter['ppn'] = 20

In [135]:
jobadmin.equilibration_queueadapter['queue'] = ''
jobadmin.equilibration_queueadapter['walltime'] = '24:00:00'
jobadmin.equilibration_queueadapter['nodes'] = 2
jobadmin.equilibration_queueadapter['ppn'] = 20

In [136]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='minimization', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [164]:
fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [9933,
  10153,
  10850,
  11326]}

In [88]:
fw_id_dict = jobadmin.identifyIncompleteWorkflowsInScope(
     precedent_step_name='minimization',
     subsequent_step_name='equilibration_nvt',
     system_name_scope=system_name_scope)

In [110]:
fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [10140,
  10837,
  11318]}

In [138]:
nvtEquilibrate_wrapper = lambda system_name: jobadmin.nvtEquilibrate(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} -v has_indenter 1',
        '-v pbd2d 0 -v reinitialize_velocities 1 -v nvtEqSteps 10000',
        '-v compute_interactions 0 -v store_forces 0',
        '-v mpiio 1 -v surfactant_name SDS')))

In [139]:
fw_dict = jobadmin.batch_prepare_fw( system_names, nvtEquilibrate_wrapper )

In [140]:
fw_dict['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'].as_dict()

{'created_on': '2019-02-04T21:07:53.458353',
 'fw_id': -9,
 'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_equilibration_nvt',
 'spec': {'_category': 'nemo_queue',
  '_files_in': {'data_file': 'datafile.lammps'},
  '_files_out': {'data_file': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_nvtEquilibrated.lammps'},
  '_queueadapter': {'nodes': 2,
   'ppn': 20,
   'queue': '',
   'walltime': '24:00:00'},
  '_tasks': [{'_fw_name': 'GetFilesTask',
    'identifiers': ['lmp_header.input',
     'lmp_equilibration_nvt.input',
     'extract_thermo.sh']},
   {'_fw_name': 'ScriptTask',
    'fizzle_bad_rc': True,
    'script': ['module purge; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-04Feb19; mpirun ${MPIRUN_OPTIONS} lmp -in lmp_equilibration_nvt.input -v baseName 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped -v dataFile datafile.lammps -v has_indenter 1 -v pbd2d 0 -v reinitialize_velocities 1 -v nvtEqStep

In [141]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [10140, 10837, 11318]


### NPT equilibration

In [115]:
# fw_id_dict = jobadmin.query_systems(
#     system_names,
#     step='minimization', 
#     state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [142]:
fw_id_dict = jobadmin.identifyIncompleteWorkflowsInScope(
     precedent_step_name='equilibration_nvt',
     subsequent_step_name='equilibration_npt',
     system_name_scope=system_name_scope)

In [143]:
fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [11322,
  11323,
  11324,
  11325]}

In [144]:
nptEquilibrate_wrapper = lambda system_name: jobadmin.nptEquilibrate(
    system_name,
    lmp_suffix_template=' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s} -v has_indenter 1',
        '-v pbd2d 0 -v reinitialize_velocities 0 -v nptEqSteps 10000',
        '-v pbc2d 0 -v compute_interactions 0 -v store_forces 0',
        '-v mpiio 0 -v surfactant_name SDS')))

In [145]:
fw_dict = jobadmin.batch_prepare_fw( system_names, nptEquilibrate_wrapper )

In [146]:
fw_dict['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'].as_dict()

{'created_on': '2019-02-04T21:09:44.544073',
 'fw_id': -10,
 'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_equilibration_npt',
 'spec': {'_category': 'nemo_queue',
  '_files_in': {'data_file': 'datafile.lammps'},
  '_files_out': {'data_file': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_nptEquilibrated.lammps'},
  '_queueadapter': {'nodes': 2,
   'ppn': 20,
   'queue': '',
   'walltime': '24:00:00'},
  '_tasks': [{'_fw_name': 'GetFilesTask',
    'identifiers': ['lmp_header.input',
     'lmp_equilibration_npt.input',
     'extract_thermo.sh']},
   {'_fw_name': 'ScriptTask',
    'fizzle_bad_rc': True,
    'script': ['module purge; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-04Feb19; mpirun ${MPIRUN_OPTIONS} lmp -in lmp_equilibration_npt.input -v baseName 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped -v dataFile datafile.lammps -v has_indenter 1 -v pbd2d 0 -v reinitialize_velocities 0 -v nptEqSte

In [147]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [11322, 11323, 11324, 11325]


### Constant velocity approach

In [19]:
d = 30 # Ang from tip apex to surface

In [5]:
dt = 2 # fs per timestep

In [6]:
# Ang / fs = 1e15 / 1e 10

In [12]:
v_SI_in_v = 1e-5 # v_SI / v = (m/s * fs/Ang)

In [14]:
v_SI = 1 # m / s

In [15]:
v = v_SI * v_SI_in_v

In [20]:
v  # Ang / fs

1e-05

In [21]:
sim_time_in_fs = d / v

In [23]:
sim_time_in_fs / 1e6 # ns

2.9999999999999996

In [24]:
total_steps = sim_time_in_fs / dt

In [25]:
total_steps

1499999.9999999998

In [26]:
totalSteps = lambda v_SI: round(d/(v_SI*v_SI_in_v*dt))

In [27]:
totalSteps(1)

1500000

In [16]:
totalSteps(10)*2*v # Ang

75.0

In [65]:
# d / v * dt

In [66]:
# 5 m /s ~ 5e-5 Ang / fs 


In [108]:
system_names

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'}

In [109]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='equilibration_npt', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [110]:
fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [9933,
  10153,
  10850,
  11326]}

In [111]:
jobadmin.use_nemo_queue()

In [112]:
jobadmin.use_nemo_templates()

In [142]:
377000/8/20

2356.25

In [126]:
# test on express queue
jobadmin.production_queueadapter['queue'] = 'express'
jobadmin.production_queueadapter['walltime'] = '00:05:00'
jobadmin.production_queueadapter['nodes'] = 1
jobadmin.production_queueadapter['ppn'] = 20

In [143]:
jobadmin.production_queueadapter['queue'] = ''
jobadmin.production_queueadapter['walltime'] = '96:00:00'
jobadmin.production_queueadapter['nodes'] = 8
jobadmin.production_queueadapter['ppn'] = 20

In [114]:
jobadmin._template_lmp_cmd # double-check usage of correct LAMMPS command

'module purge; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-09Feb19; mpirun ${{MPIRUN_OPTIONS}} lmp -in {inputFile:s}'

In [29]:
v_SI = 0.1

In [30]:
print("{:f}".format(v_SI*v_SI_in_v))

0.000001


In [34]:
totalSteps(v_SI) * 2e-15 /1e-9

75.0

In [32]:
# netcdf no of frames:
totalSteps(v_SI) / 10000

3750.0

In [155]:
# Attention: negative velocity for approach towards substrate!
production_wrapper = lambda system_name: jobadmin.production(
    system_name,
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s}',
        '-v has_indenter 1 -v pbd2d 0',
        '-v constant_indenter_velocity {:f}'.format(-v_SI*v_SI_in_v),
        '-v reinitialize_velocities 0',
        '-v productionSteps {:d}'.format(totalSteps(v_SI)),
        '-v use_colvars 0 -v mpiio 1',
        '-v thermo_frequency 1000',
        '-v thermo_average_frequency 1000',
        '-v netcdf_frequency 10000',
        '-v compute_interactions 1',
        '-v store_forces 1',
        '-v surfactant_name SDS')),
    total_steps=totalSteps(v_SI)
)

In [156]:
fw_dict = jobadmin.batch_prepare_fw( system_names, production_wrapper )

In [157]:
fw_dict['646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped'].as_dict()

{'created_on': '2019-02-15T14:50:51.170131',
 'fw_id': -6,
 'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_produtcion_3750000',
 'spec': {'_category': 'nemo_queue',
  '_files_in': {'data_file': 'datafile.lammps'},
  '_files_out': {'data_file': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_production.lammps',
   'thermo_file': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_production_thermo.out',
   'trajectory_file': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_production_mixed.nc'},
  '_queueadapter': {'nodes': 8,
   'ppn': 20,
   'queue': '',
   'walltime': '96:00:00'},
  '_tasks': [{'_fw_name': 'GetFilesTask',
    'identifiers': ['lmp_header.input',
     'lmp_production.input',
     'lmp_production_mixed.input',
     'extract_thermo.sh']},
   {'_fw_name': 'ScriptTask',
    'fizzle_bad_rc': True,
    'script': ['module purge; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvar

In [158]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [9933, 10153, 10850, 11326]


### Colvars

In [ ]:
system_names

In [525]:
fw_id_dict = jobadmin.query_systems(
    system_names,
    step='equilibration_npt', 
    state=["COMPLETED", "RUNNING", "READY", "WAITING", "RESERVED"])

In [523]:
fw_id_dict = jobadmin.identifyIncompleteWorkflowsInScope(
     precedent_step_name='equilibration_npt',
     subsequent_step_name='production',
     system_name_scope=system_name_scope)

In [ ]:
system_names = [ system_name for system_name, fw_ids in fw_id_dict.items() if len(fw_ids) > 0 ]

In [527]:
jobadmin.production_queueadapter['nodes'] = 8

In [528]:
jobadmin.production_queueadapter['walltime'] = '96:00:00'

In [ ]:
jobadmin._template_lmp_cmd # double-check usage of correct LAMMPS command

In [ ]:
# 500,000 steps ~ 1 ns
# 10 nm / 1 ns ~ 10 m / s

# 1,000,000 steps ~ 2 ns
# 10 nm / 2 ns ~ 5 m / s

In [532]:
force_constant=25000
approach_steps = 500000
total_steps = int(1.1*approach_steps)

In [533]:
production_wrapper = lambda system_name: jobadmin.colvars_production(
    system_name,
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s} -v dataFile {dataFile:s}',
        '-v has_indenter 1 -v pbd2d 0',
        '-v reinitialize_velocities 0',
        '-v productionSteps {:d}'.format(totalSteps(10)),
        '-v use_colvars 1 -v mpiio 0',
        '-v thermo_frequency 10',
        '-v netcdf_frequency 1000',
        '-v compute_interactions 1',
        '-v store_forces 1',
        '-v surfactant_name SDS')),
    force_constant=force_constant,
    total_steps=approach_steps,
    initial_distance = 100.0, # nm, COM-COM
    final_distance   = 10.0)

In [534]:
fw_dict = jobadmin.batch_prepare_fw( system_names, production_wrapper )

In [535]:
jobadmin.append_wf_by_key(fw_dict, fw_id_dict)

Append 1107_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10330, 10838]
Append 377_SDS_on_AU_111_51x30x2_cylinders_with_counterion_50Ang_stepped to last fw_id in list [10149, 10839]
Append 220_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [9930, 10150, 10840]
Append 129_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [9928, 10148, 10841]
Append 75_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10326, 10842]
Append 646_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [10327, 10843]
Append 220_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [9926, 10146, 10844]
Append 75_SDS_on_AU_111_51x30x2_bilayer_with_counterion_50Ang_stepped to last fw_id in list [10328, 10845]
Append 377_SDS_on_AU_111_51x30x2_monolayer_with_counterion_50Ang_stepped to last fw_id in list [9925, 10145, 

### Continuation tasks

In [46]:
import pymongo

In [86]:
total_steps = 500000

In [87]:
completed_fw_ids = jobadmin.lpad.get_fw_ids(
    query = { 
        'state':            "COMPLETED",
        'spec.system_name':  { '$in': system_name_scope },
        'spec.step':        'production',
        'spec.total_steps': total_steps },
    sort = [('updated_on', pymongo.ASCENDING)])
             

In [88]:
completed_fw_ids

[10856, 10858, 10860, 10854, 10861, 10862, 10855, 10851, 10857, 10859, 10853]

In [89]:
completed_fw_id_dict = {}
for fw_id in completed_fw_ids:
    fw = jobadmin.lpad.get_fw_by_id(fw_id)
    if fw.spec['system_name'] in completed_fw_id_dict:
        completed_fw_id_dict[ fw.spec['system_name'] ].append(fw_id)
    else:
        completed_fw_id_dict[ fw.spec['system_name'] ] = [ fw_id ]
    

In [90]:
fizzled_fw_ids = jobadmin.lpad.get_fw_ids(
    query = { 
        'state':            "FIZZLED",
        'spec.system_name':  { '$in': system_name_scope },
        'spec.step':        'production',
        'spec.total_steps':      total_steps },
    sort = [('updated_on', pymongo.ASCENDING)])
             

In [91]:
fizzled_fw_ids

[10225, 10226, 10232, 10231]

In [62]:
fizzled_fw_id_dict = {}
for fw_id in fizzled_fw_ids:
    fw = jobadmin.lpad.get_fw_by_id(fw_id)
    if fw.spec['system_name'] in fizzled_fw_id_dict:
        fizzled_fw_id_dict[ fw.spec['system_name'] ].append(fw_id)
    else:
        fizzled_fw_id_dict[ fw.spec['system_name'] ] = [ fw_id ]
    

In [66]:
discontinued_systems = fizzled_fw_id_dict.keys() - completed_fw_id_dict.keys()

In [67]:
discontinued_fw_id_dict = { 
    system_name: fizzled_fw_id_dict[system_name] for system_name in discontinued_systems }

In [68]:
discontinued_fw_id_dict

{'646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped': [10226,
  10232]}

In [73]:
system_names = list( discontinued_fw_id_dict.keys() )

In [42]:
system_names

NameError: name 'system_names' is not defined

In [69]:
jobadmin.production_queueadapter

{'nodes': 4, 'walltime': '96:00:00'}

In [70]:
jobadmin.production_queueadapter['nodes'] = 8

In [71]:
force_constant=25000
approach_steps = 500000
total_steps = int(1.1*approach_steps)

In [72]:
restart_wrapper = lambda system_name: jobadmin.prepare_restart(
    system_name,
    lmp_suffix_template= ' '.join((
        '-v baseName {baseName:s}',
        '-v has_indenter 1 -v pbd2d 0',
        '-v reinitialize_velocities 0',
        '-v productionSteps {:d}'.format(total_steps),
        '-v use_colvars 1 -v mpiio 0',
        '-v thermo_frequency 10',
        '-v netcdf_frequency 1000',
        '-v compute_interactions 1',
        '-v store_forces 1',
        '-v surfactant_name SDS')),
    output_subfolder='production_{:d}'.format(approach_steps))

In [75]:
fw_dict = jobadmin.batch_prepare_fw( system_names, restart_wrapper )

In [76]:
fw_dict[system_names[0]].as_dict()

{'created_on': '2018-10-28T13:36:12.622599',
 'fw_id': -1,
 'name': '646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped_recover_lammps',
 'spec': {'_allow_fizzled_parents': True,
  '_category': 'nemo_noqueue',
  '_tasks': [{'_fw_name': 'RecoverLammpsTask',
    'continue': True,
    'dest': '/work/ws/nemo/fr_jh1130-201708-0/jobs/lmplab/sds/201809/sys/646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped/production_500000',
    'file_transfer_mode': 'copy',
    'fizzle_on_no_restart_file': True,
    'lmp_cmd': 'module purge; module use ${HOME}/modulefiles; module load lammps/16Mar18-gnu-7.3-openmpi-3.1-colvars-20Sep18; mpirun ${MPIRUN_OPTIONS} lmp -in lmp_production.input',
    'lmp_opt': '-v baseName {baseName:s} -v has_indenter 1 -v pbd2d 0 -v reinitialize_velocities 0 -v productionSteps 550000 -v use_colvars 1 -v mpiio 0 -v thermo_frequency 10 -v netcdf_frequency 1000 -v compute_interactions 1 -v store_forces 1 -v surfactant_name SDS',
    'lmp_queue

In [77]:
jobadmin.append_wf_by_key(fw_dict, discontinued_fw_id_dict)

Append 646_SDS_on_AU_111_51x30x2_hemicylinders_with_counterion_50Ang_stepped to last fw_id in list [10226, 10232]


## Progress check & recovery 

Manually append "post LAMMPS" fireworks to fizzled production runs,
in order to run in same directory as previous and pull already produced output files.

In [ ]:
system_name_scope

In [ ]:
fizzled_production_dict = jobadmin.query_step(
    step='production', state_list = ['FIZZLED'] )

In [ ]:
fizzled_production_dict

In [ ]:
fw_ids = jobadmin.lpad.get_fw_ids(
    query = { 
        "spec.step":      "production",
        "state":          "FIZZLED", 
        "spec._category": "nemo_queue"} )

In [ ]:
fw_id_system_name_dict = { 
    fw_id: jobadmin.lpad.get_fw_by_id(fw_id).spec["system_name"] for fw_id in fw_ids }

In [ ]:
system_names = list(
    set(fw_id_system_name_dict.values()) & set(system_name_scope) )

In [ ]:
fw_id_dict = { 
    jobadmin.lpad.get_fw_by_id(fw_id).name: fw_id \
    for fw_id, system_name in fw_id_system_name_dict.items() \
    if system_name in system_names }

In [ ]:
fw_id_dict

In [ ]:
fw_dict = {}
for fw_name, fw_id in fw_id_dict.items():
    launch_dir = jobadmin.lpad.get_launchdir(fw_id)
    fw_spec = jobadmin.lpad.get_fw_by_id(fw_id).spec
    system_name = fw_spec["system_name"]
    if "total_steps" in fw_spec:
        total_steps = fw_spec["total_steps"]
    else:
        total_steps = 1000000
        
    step = fw_spec["step"]
    fw_dict[fw_name] = jobadmin.post_lammps_fw(
        system_name = system_name, 
        parent_fw_id = fw_id, 
        launch_dir = launch_dir,
        output_folder = '{:s}_{:d}'.format(step,total_steps),
        step= 'production', lmp_suffix='_production_mixed')

In [ ]:
fw_dict[ list(fw_dict.keys())[0] ].as_dict()

In [ ]:
for fw_name, fw in fw_dict.items():
    jobadmin.lpad.append_wf(
        new_wf = Workflow( [ fw ] ), 
        fw_ids = [ fw_id_dict[ fw_name ] ] )

In [ ]:
fw_dict[ list(fw_dict.keys())[0] ]

### Defuse certain fw ids

In [ ]:
for fw_id in fw_ids: jobadmin.lpad.defuse_fw( fw_id, rerun_duplicates=False )

### Rerun certain fw ids

In [ ]:
for key, fw_ids in fw_id_dict.items():
    jobadmin.lpad.rerun_fw(fw_ids[-1],rerun_duplicates=False)

In [ ]:
jobadmin.lpad.rerun_fw(9922,rerun_duplicates=False)

### Detect certain lost runs
fizzle and forget them.
For some reason, offline runs are un-fizzled again when recovered if not forcefully "forgotten"

In [ ]:
launch_ids, fw_ids, _ = \
    jobadmin.lpad.detect_lostruns(expiration_secs=1, query= {
        'spec.system_name': { '$regex': '.*CTAB.*'}
    },fizzle=True)
for fw_id in fw_ids: jobadmin.lpad.forget_offline(fw_id)    

In [ ]:
# detect lost runs except certain fw ids
# do not recover them afterwards
launch_ids, fw_ids, _ = \
    jobadmin.lpad.detect_lostruns(expiration_secs=1, query= {
        'fw_id': {'$nin': [ 10232,10231 ] }
    }, fizzle=True)
for fw_id in fw_ids: jobadmin.lpad.forget_offline(fw_id)

In [350]:
3 +4*4

19

In [ ]:
3+